In [1]:
import gc
import os
import sys
import h5py
import random
import json
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

package_path = Path(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
sys.path.insert(0, str(package_path))

In [2]:
from src.neuronio.neuronio_data_utils import (
    DEFAULT_Y_SOMA_THRESHOLD,
    DEFAULT_Y_TRAIN_SOMA_BIAS,
    DEFAULT_Y_TRAIN_SOMA_SCALE,
    NEURONIO_DATA_DIM,
    NEURONIO_LABEL_DIM,
    NEURONIO_SIM_LEN,
    NEURONIO_SIM_PER_FILE,
    create_neuronio_input_type,
    parse_sim_experiment_file,
    get_data_files_from_folder, 
)

In [3]:
# General Config
general_config = dict()
general_config["seed"] = 0
general_config["device"] = 'cuda' if torch.cuda.is_available() else 'cpu'
general_config["train_valid_eval"] = False
general_config["artefacts_dir"]= "../models/new_exp/forget_True_rest_True_nummem_10"
torch_device = torch.device(general_config["device"])
print("Torch Device: ", torch_device)

Torch Device:  cuda


In [4]:
# Seeding & Determinism
os.environ['PYTHONHASHSEED'] = str(general_config["seed"])
random.seed(general_config["seed"])
np.random.seed(general_config["seed"])
torch.manual_seed(general_config["seed"])
torch.cuda.manual_seed(general_config["seed"])
torch.backends.cudnn.deterministic = True

In [5]:
data_dir_path = Path("D:/NeuronIO").expanduser().resolve() # TODO: change to neuronio data path
train_data_dir_path = data_dir_path / "train"  # TODO: change to train subfolder
test_data_dir_path = data_dir_path / "test/Data_test"  # TODO: change to test subfolder

data_config = dict()
train_data_dirs = [
    str(train_data_dir_path / "full_ergodic_train_batch_1"),
    str(train_data_dir_path / "full_ergodic_train_batch_2"),
    str(train_data_dir_path / "full_ergodic_train_batch_3"),
    str(train_data_dir_path / "full_ergodic_train_batch_4"),
    str(train_data_dir_path / "full_ergodic_train_batch_5"),
    str(train_data_dir_path / "full_ergodic_train_batch_6"),
    str(train_data_dir_path / "full_ergodic_train_batch_7"),
    str(train_data_dir_path / "full_ergodic_train_batch_8"),
    str(train_data_dir_path / "full_ergodic_train_batch_9"),
    str(train_data_dir_path / "full_ergodic_train_batch_10"),
]
test_data_dirs = [str(test_data_dir_path)]

data_config["train_data_dirs"] = train_data_dirs
data_config["test_data_dirs"] = test_data_dirs

data_config["data_dim"] = NEURONIO_DATA_DIM 
data_config["label_dim"] = NEURONIO_LABEL_DIM

train_files = get_data_files_from_folder(data_config["train_data_dirs"])
test_files = get_data_files_from_folder(data_config["test_data_dirs"])

## Load model

In [6]:
from src.expressive_leaky_memory_neuron import ELM
from src.expressive_leaky_memory_neuron_forget import ELMf
from src.neuronio.neuronio_data_utils import (
    NEURONIO_DATA_DIM, 
    NEURONIO_LABEL_DIM, 
    get_data_files_from_folder, 
    parse_sim_experiment_file,
    visualize_training_batch,
)
from src.neuronio.neuronio_data_loader_filtered import NeuronIO
from src.neuronio.neuronio_train_utils import NeuronioLoss
from src.neuronio.neuronio_eval_utils_filtered import (
    NeuronioEvaluator, 
    compute_test_predictions_multiple_sim_files, 
    filter_and_extract_core_results,
)
from src.neuronio.neuronio_data_utils import parse_sim_experiment_file, create_neuronio_input_type, DEFAULT_Y_SOMA_THRESHOLD
from src.neuronio.neuronio_viz_utils import visualize_neuron_workings

In [7]:
# load model config
with open(general_config["artefacts_dir"] + "/model_config.json") as f:
    model_config = json.load(f)

# load train config
with open(general_config["artefacts_dir"] + "/train_config.json") as f:
    train_config = json.load(f)

In [8]:
# Initialize the ELM model
if train_config['forget_gate']:
    model = ELMf(**model_config).to(torch_device)
else:
    model = ELM(**model_config).to(torch_device)


# Load the best model for evaluation
state_dict = torch.load(general_config["artefacts_dir"] + "/neuronio_best_model_state.pt", map_location=torch_device)
print(model.load_state_dict(state_dict))
# Visualize ELM model
print(model)

<All keys matched successfully>
ELMf(
  (mlp): RecursiveScriptModule(
    original_name=MLP
    (network): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=ReLU)
      (2): RecursiveScriptModule(original_name=Linear)
    )
  )
  (mlpf): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=Sigmoid)
  )
  (w_y): RecursiveScriptModule(original_name=Linear)
)


C:\Users\Bingkun LIU\AppData\Local\Temp\ipykernel_11300\3546985411.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(general_config["artefacts_dir"

In [9]:
# apples to apples comparison
sample_file_name = "100523.p"
sample_sequence_index = 33
sample_select_start = 2000
sample_select_len = 2000

# load file
select_test_file = [file for file in test_files if sample_file_name in file][0]
viz_X, viz_y_spike, viz_y_soma = parse_sim_experiment_file(select_test_file)

# select sequence
viz_X = viz_X[:, sample_select_start: sample_select_start+sample_select_len, sample_sequence_index]
viz_y_spike = viz_y_spike[sample_select_start: sample_select_start+sample_select_len, sample_sequence_index]
viz_y_soma = viz_y_soma[sample_select_start: sample_select_start+sample_select_len, sample_sequence_index]

# preprocess sequence
synapse_types = create_neuronio_input_type()
viz_X = torch.tensor(viz_X.T * synapse_types[:]).to(torch_device)
viz_y_soma = np.clip(viz_y_soma, a_min=None, a_max=DEFAULT_Y_SOMA_THRESHOLD)

KeyboardInterrupt: 

In [25]:
viz_X

tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0, -1,  0]], device='cuda:0', dtype=torch.int32)

In [27]:
train_config

{'batch_size': 8,
 'batches_per_epoch': 10000,
 'burn_in_time': 150,
 'file_load_fraction': 0.3,
 'forget_gate': True,
 'ignore_time_from_start': 400,
 'input_window_size': 500,
 'learning_rate': 0.0005,
 'num_epochs': 35,
 'num_prefetch_batch': 20,
 'num_workers': 6,
 'rest_start': True,
 'sec_len': 200,
 'start_save_path': '../data_processed/NeuronIOstartpoint/'}

In [26]:
visualize_neuron_workings(
    neuron=model,
    input_spikes=viz_X,
    target_spikes=viz_y_spike,
    target_soma=viz_y_soma,
    color_by_memory_tau=False
)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

## Evaluation with different burn-in

In [9]:
train_config['burn_in_time']
train_config['rest_start']
burn_in_times = np.arange(0, 200, 10)

In [10]:
# Gather all evaluation metrics
# set burn-in in 
with torch.no_grad():
    if general_config["train_valid_eval"]:
        random.seed(general_config["seed"])
        select_train_files = random.choices(train_files, k=10)
        train_predictions = compute_test_predictions_multiple_sim_files(
            neuron=model,
            test_files=select_train_files,
            burn_in_time=0,
            input_window_size=train_config["input_window_size"],
            rest_start = train_config['rest_start'],
            device=torch_device,
        )
        train_results = filter_and_extract_core_results(*train_predictions, burn_in_times=burn_in_times, verbose=False)

        valid_predictions = compute_test_predictions_multiple_sim_files(
            neuron=model,
            test_files=valid_files,
            burn_in_time=0,
            input_window_size=train_config["input_window_size"],
            rest_start = train_config['rest_start'],
            device=torch_device,
        )
        valid_results = filter_and_extract_core_results(*valid_predictions, burn_in_times=burn_in_times, verbose=False)

    test_predictions = compute_test_predictions_multiple_sim_files(
        neuron=model,
        test_files=test_files,
        burn_in_time=0,
        input_window_size=train_config["input_window_size"],
        rest_start = train_config['rest_start'],
        device=torch_device,
    )
    test_results = filter_and_extract_core_results(*test_predictions, burn_in_times=burn_in_timse, verbose=False)

eval_results = dict()
if general_config["train_valid_eval"]:
    eval_results["train_results"] = train_results
    eval_results["valid_results"] = valid_results
eval_results["test_results"] = test_results

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# Show evaluation results
eval_results

## Preprocess starting points NeuronIO

In [4]:
ignore_time_from_start = 500
threshold = -65
threshold_h = -60
section_len = 200
start_point_persec = 20

In [ ]:
save_path = '../data_processed/NeuronIOstartpoint/'
for file_path in test_files:
    X, y_spike, y_soma = parse_sim_experiment_file(
        sim_experiment_file=file_path,
        include_params=False,
        )
    sim_len, sim_num = y_soma.shape
    num_sec = int(sim_len/section_len)
    sections = y_soma.T.reshape([sim_num,num_sec,section_len])
    start_time  = np.ones([sim_num, num_sec, start_point_persec], dtype=int)
    for sim in range(sim_num):
        for i in range(num_sec):
            choice = (sections[sim, i, :] < threshold).nonzero()[0]
            if len(choice) < start_point_persec:
                choice = (sections[sim, i, :] < threshold_h).nonzero()[0]
            try:
                start_time[sim, i, :] = np.random.choice(choice, start_point_persec, replace=False)
            except:
                start_time[sim, i, :] = np.random.choice(choice, start_point_persec, replace=True)
                print(file_path[-93:-2])
                print(len(choice))
            start_time[sim, i, :] += i * section_len
    
    start_time = start_time.reshape([sim_num, -1])
    np.save(save_path + file_path[-92:-2]+'.npy', start_time, allow_pickle=True)
    #print(file_path[-93:-2])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000249380EBBB0>>
Traceback (most recent call last):
  File "C:\Anaconda3\envs\elm\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [149]:
for sim in range(sim_num):
    print(y_soma[start_time[sim], sim].max())

-65.0625
-65.0625
-65.0625
-65.0625
-65.0625
-65.125
-60.15625
-65.1875
-65.0625
-65.0625
-65.0625
-60.40625
-65.125
-60.09375
-65.125
-65.0625
-65.375
-65.0625
-65.5
-65.0625
-60.0625
-65.0625
-65.1875
-65.0625
-65.0625
-65.0625
-60.03125
-65.375
-65.125
-60.5
-60.03125
-65.25
-60.03125
-60.0625
-65.3125
-65.0625
-60.03125
-60.0625
-60.125
-65.125
-65.0625
-65.0625
-65.0625
-60.0625
-60.0625
-65.0625
-65.0625
-65.0625
-65.0625
-65.1875
-65.9375
-65.0625
-65.0625
-60.28125
-65.0625
-65.0625
-65.0625
-60.15625
-65.0625
-65.0625
-65.125
-65.0625
-65.1875
-65.8125
-65.1875
-60.375
-60.03125
-65.125
-65.3125
-65.1875
-60.0625
-65.0625
-60.03125
-65.5
-65.0625
-65.0625
-60.9375
-60.15625
-65.125
-65.125
-60.0625
-60.625
-60.125
-60.5
-60.03125
-65.4375
-65.0625
-60.125
-60.125
-65.0625
-65.0625
-65.375
-65.1875
-65.0625
-60.03125
-69.5
-60.03125
-60.4375
-65.0625
-60.5
-65.125
-65.0625
-65.0625
-65.0625
-60.03125
-60.0625
-65.3125
-65.0625
-60.1875
-60.09375
-60.5
-65.0625
-60.96875
-60.531

## Generating Random Input

## Generating Output from random input

In [ ]:
from src.neuronio.neuronio_eval_utils import (
    NeuronioEvaluator, 
    compute_test_predictions_multiple_sim_files, 
    filter_and_extract_core_results,
)

with torch.no_grad():
    test_predictions = compute_test_predictions_multiple_sim_files(
        neuron=model,
        test_files=test_files,
        burn_in_time=train_config["burn_in_time"],
        input_window_size=train_config["input_window_size"],
        device=torch_device,
    )
    test_results = filter_and_extract_core_results(*test_predictions, verbose=False)

eval_results = dict()
if general_config["train_valid_eval"]:
    eval_results["train_results"] = train_results
    eval_results["valid_results"] = valid_results
eval_results["test_results"] = test_results